## MVO Weights
4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,
2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,
7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245


In [1]:
import pandas as pd
import numpy as np
import random
import math
from datetime import date,timedelta

In [2]:
def BALANCE(weights):
  #Making sure the total sum of the weights eual to 1
  weights = [w/sum(weights) for w in weights] 
  # Making sure all weights represent proportions that add up to 1
  return weights

In [3]:
def ratio(a,b,c):                       
  #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  #calculating sharpe ratio
  return (a-c)/b

In [4]:
# def number_of_years(y):#calculates the number of years of the dataset
#   p=y.index[0]         #date of first row in the dataset (datetime format)
#   q=y.index[len(y)-1]  #date of last row in the dataset  (datetime format)
#   return ((q-p).days+1)/365

In [5]:
df=pd.read_csv("2021-01-14.csv",parse_dates=['Date'],index_col='Date')  #Importing Dataset
#df = df.loc["2016-01-01" :"2020-12-31" ]   #Since 2016-01-01, 5y(1234rows till 2020-12-31)
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)
col=list(df.columns)
tdf.shape

(3, 24)

In [6]:
end=tdf.index[2].date()

In [7]:
trading_days=250#len(df)/number_of_years(tdf) 

In [8]:
returnsh=df.pct_change()                  
#Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)

In [9]:
returnsh=returnsh.iloc[1:,:]
returnsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,-0.006194,-0.027627,-0.018456,-0.002891,-0.012767,0.022815,-0.005790,-0.006988,-0.006431,-0.001494,...,0.001122,0.006382,-0.009325,-0.005008,-0.006247,0.020506,0.005121,-0.013948,0.002084,0.002840
2,-0.014721,-0.005211,-0.005265,0.012234,0.001425,-0.012677,-0.026753,-0.001292,-0.004759,0.011044,...,0.006288,0.000488,0.011244,0.031003,-0.010316,-0.003948,-0.015940,-0.010526,-0.012105,-0.010131


In [10]:
returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
  for i in range(1, len(df)):
    if returnso[cols][i] > 0:
      returnso[cols][i] = 0


In [11]:
returnso

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
1,-0.006194,-0.027627,-0.018456,-0.002891,-0.012767,0.000000,-0.005790,-0.006988,-0.006431,-0.001494,...,0.0,0.0,-0.009325,-0.005008,-0.006247,0.000000,0.00000,-0.013948,0.000000,0.000000
2,-0.014721,-0.005211,-0.005265,0.000000,0.000000,-0.012677,-0.026753,-0.001292,-0.004759,0.000000,...,0.0,0.0,0.000000,0.000000,-0.010316,-0.003948,-0.01594,-0.010526,-0.012105,-0.010131


In [12]:
covmatsh=returnsh.cov()*trading_days     
#Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso = returnso.cov() * trading_days

In [13]:
covmatsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
ASIANPAINT,0.009089,-0.023892,-0.014060,-0.016121,-0.015127,0.037830,0.022344,-0.006072,-0.001782,-0.013364,...,-0.005506,0.006282,-0.021924,-0.038383,0.004337,0.026066,0.022448,-0.003647,0.015123,0.013825
BAJFINANCE,-0.023892,0.062808,0.036961,0.042380,0.039766,-0.099448,-0.058737,0.015961,0.004684,0.035131,...,0.014473,-0.016515,0.057634,0.100901,-0.011402,-0.068522,-0.059012,0.009588,-0.039755,-0.036344
BAJAJFINSV,-0.014060,0.036961,0.021751,0.024939,0.023401,-0.058522,-0.034565,0.009393,0.002756,0.020674,...,0.008517,-0.009719,0.033916,0.059377,-0.006710,-0.040323,-0.034727,0.005642,-0.023395,-0.021388
BRITANNIA,-0.016121,0.042380,0.024939,0.028596,0.026832,-0.067102,-0.039633,0.010770,0.003160,0.023705,...,0.009766,-0.011143,0.038889,0.068083,-0.007693,-0.046235,-0.039818,0.006470,-0.026825,-0.024523
DIVISLAB,-0.015127,0.039766,0.023401,0.026832,0.025177,-0.062964,-0.037188,0.010106,0.002966,0.022243,...,0.009163,-0.010456,0.036490,0.063884,-0.007219,-0.043384,-0.037363,0.006071,-0.025170,-0.023011
GRASIM,0.037830,-0.099448,-0.058522,-0.067102,-0.062964,0.157461,0.093001,-0.025273,-0.007416,-0.055625,...,-0.022916,0.026149,-0.091256,-0.159762,0.018053,0.108495,0.093437,-0.015181,0.062947,0.057546
HCLTECH,0.022344,-0.058737,-0.034565,-0.039633,-0.037188,0.093001,0.054929,-0.014927,-0.004380,-0.032854,...,-0.013535,0.015444,-0.053898,-0.094360,0.010662,0.064080,0.055187,-0.008967,0.037178,0.033988
HDFCBANK,-0.006072,0.015961,0.009393,0.010770,0.010106,-0.025273,-0.014927,0.004056,0.001190,0.008928,...,0.003678,-0.004197,0.014647,0.025642,-0.002897,-0.017413,-0.014997,0.002437,-0.010103,-0.009236
HINDALCO,-0.001782,0.004684,0.002756,0.003160,0.002966,-0.007416,-0.004380,0.001190,0.000349,0.002620,...,0.001079,-0.001232,0.004298,0.007525,-0.000850,-0.005110,-0.004401,0.000715,-0.002965,-0.002710
HINDUNILVR,-0.013364,0.035131,0.020674,0.023705,0.022243,-0.055625,-0.032854,0.008928,0.002620,0.019650,...,0.008095,-0.009237,0.032237,0.056438,-0.006377,-0.038327,-0.033008,0.005363,-0.022237,-0.020329


In [14]:
risk_free_rate = 0.0358 #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [15]:
df
stocks=df.shape[1]
stocks

24

# Sharpe


In [16]:
global_warr_sortino=[]
global_war_sharpe=[]
sharpe_portfolio_return=[]
sharpe_portfolio_risk=[]
sharpe_portfolio_shratio=[]
sharpe_portfolio_stockWeights=[]

In [17]:

fitness=0
#for each iteration

weights=np.array(BALANCE(np.array([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
])))

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varsh=np.dot(weights.T,np.dot(covmatsh,weights))   

#portfolio risk
volatility_temp = np.sqrt(varsh)      

#Calculating fitness value(ie sharpe ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sharpe_pbest)


sharpe_portfolio_return.append(returns_temp)
sharpe_portfolio_risk.append(volatility_temp)
sharpe_portfolio_shratio.append(fitness)
sharpe_portfolio_stockWeights.append(weights)



In [18]:
sharpe_portfolio = {'Returns' : sharpe_portfolio_return, 'Standard Deviation' : sharpe_portfolio_risk,  'Sharpe Ratio' : sharpe_portfolio_shratio}  

for counter,symbol in enumerate(df.columns):
  sharpe_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sharpe_portfolio_stockWeights]
sharpe_pc = pd.DataFrame(sharpe_portfolio)
sharpe_optimal=sharpe_pc.iloc[sharpe_pc['Sharpe Ratio'].idxmax()]
sharpe_optimal=pd.DataFrame(sharpe_optimal)
sharpe_optimal.to_csv("sharpe_optimal.csv")

sharpe_pc

,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,-0.50257,0.064272,-8.376392,0.045761,0.09677,0.028816,0.037628,0.061001,0.011471,0.0164,...,0.097623,0.020142,0.07839,0.077287,0.022779,0.033323,0.010316,0.072171,0.006063,0.017363


In [19]:
sharpe_pc.to_csv(str(end)+'MVOsharpe_A.csv')

# Sortino

In [20]:
global_warr_sortino=[]
sortino_portfolio_return=[]
sortino_portfolio_risk=[]
sortino_portfolio_soratio=[]
sortino_portfolio_stockWeights=[]

In [21]:

fitness=0
#for each iteration


#Making sure that the total amount of pheromone equals 1 
weights=np.array(BALANCE([4.57606371261345,9.67698598407907,2.8816471398668,3.76280675093989,6.10005496604961,1.14708159257701,1.63996379334881,3.36411165704113,2.00796787062891,9.13229658964017,1.37274941345458,2.38654278474067,6.25880838891221,2.14733058728827,9.76228755822564,2.01419193025687,7.83900398874845,7.72870344184099,2.27786673057532,3.33226384976688,1.03163727917332,7.21709695950646,0.606271511043036,1.73626551968245
]))

#calculating annulaised portfolio return
returns_temp = np.sum(returnsh.mean()*weights)*trading_days 

#calculating portfolio varience wrt calculating sharpe ratio
varso=np.dot(weights.T,np.dot(covmatso,weights))   

#portfolio risk
volatility_temp = np.sqrt(varso)      

#Calculating fitness value(ie sortino ratio)
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)

#Initializing the intial fitness value as the best fitness value(sortino_pbest)

sortino_portfolio_return.append(returns_temp)
sortino_portfolio_risk.append(volatility_temp)
sortino_portfolio_soratio.append(fitness)
sortino_portfolio_stockWeights.append(weights)


In [22]:
sortino_portfolio = {'Returns' : sortino_portfolio_return, 'Semi Deviation' : sortino_portfolio_risk,  'Sortino Ratio' : sortino_portfolio_soratio}  

for counter,symbol in enumerate(df.columns):
  sortino_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sortino_portfolio_stockWeights]
sortino_pc = pd.DataFrame(sortino_portfolio)
sortino_optimal=sortino_pc.iloc[sortino_pc['Sortino Ratio'].idxmax()]
sortino_optimal=pd.DataFrame(sortino_optimal)
sortino_optimal.to_csv("sortino_optimal.csv")
sortino_pc


,Returns,Semi Deviation,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,GRASIM Weight,HCLTECH Weight,...,NESTLEIND Weight,POWERGRID Weight,RELIANCE Weight,TCS Weight,TATACONSUM Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,ULTRACEMCO Weight,WIPRO Weight
0,-0.50257,0.025306,-21.274769,0.045761,0.09677,0.028816,0.037628,0.061001,0.011471,0.0164,...,0.097623,0.020142,0.07839,0.077287,0.022779,0.033323,0.010316,0.072171,0.006063,0.017363


In [23]:

sortino_pc.to_csv(str(end)+'MVOsortino_A.csv')